# Színváltoztató LED

A LED-ek alap karakterisztikája, hogy közel egy hullámhosszon sugároznak, így lényegében csak egy színt állítanak elő (amennyiben a környezeti tényezők, mint pl. a hőmérséklet, állandó). Magában ez a tulajdonság rengeteg felhasználási lehetőséget teremt, de felmerült az igény a fehér fényű LED-re is, mivel akár stabil és hatékony fényforrás lehet.

Az egyik módja, hogy ilyen LED-et készítsenek az RGB LED, ahol lényegében egy piros (Red), egy zöld (Green) és egy kék (Blue) LED-et tesznek egy platformra és az általuk kisugárzott fény intenzitásának aránya fogja meghatározni, milyen színt fogunk látni. Szinte bármilyen színt beállíthatunk az RGB komponensek intenzitásának szabályozásával.

## Mit fogsz készíteni?

Egy RGB LED-ből és három ellenállásból álló áramkört fogunk készíteni, amivel megtanulhatjuk az RGB LED vezérlését.

## Mit tanulsz meg?

Az RGB LED áramkör elkészítésével a következőket tanulod meg:

* Hogyan tudsz egy egyszerű áramkört összerakni egy breadboard, RGB LED, ellenállások és drótok segítségével
* Hogyan programozd be a Raspberry Pi GPIO kimeneteleit a **gpiozero** modullal
* Hogyan használjuk az *RGBLED* objektumokat és változtasd azoknak a színét.

## A projekt részletekre bontása

* Inicializálni az RGB LED objektumot
* Tesztelni a LED parancsokat 

## Áramköri elemek listája

a) [Raspberry PI](https://malnapc.hu/yis/raspberry-pi/rpi-panelek) 

b) [Jumper wires female/male](https://www.ret.hu/shop/product/e-call/jumper-vezetek-szet_53-22-63) 

c) RGB LED, $I_{max}$ = 20 mA-es: [itt vásárolhatsz](https://www.tme.eu/hu/details/osta56a1a-c/tht-led-diodak-egyeb/optosupply/)

d) 3db 220 - 560 Ohm közötti [ellenállás](https://www.tme.eu/hu/katalog/tht-metal-film-ellenallasok-0-6w_100289/?s_order=desc&search=ellenallas&s_field=1000011)

e) 1db [Breadboard](https://www.tme.eu/hu/katalog/muhelyfelszereles_112607/?s_order=desc&search=breadboard&s_field=1000011)

## A kapcsolási rajz

<img src="prog04_circuit.png" width=600 height=400 />

A fenti ábrához hasonlóan kapcsoljuk össze az áramköri elemeket és a Raspberry Pi-t.

1) Helyezzük a piros LED lábait két különboző sorba. A katódjának (negatív láb) sorába kössük be az ellenállásunk egyik lábát, míg a másikat kössük a földelésre. A LED anódját (pozitív láb) egyelőre szabadon hadjuk. A LED anódjának és katódjának megállapítására használhatunk egy multimétert. Csatlakoztasd a fekete mérőzsinórt a COM (föld) és a piros mérőzsinórt a VΩmA jelzésű hüvelybe. Forgasd el a méréshatárváltó kapcsolót a folytonosság mérés állapotba. Érintsd a két zsínórt a két lábhoz és ha a LED halványan pislákol, akkor az a láb amelyikhez a fekete zsinórt érintetted a katód, a másik az anód. Ha nem pislákol akkor cseréld meg a zsinórok és a lábak érintkezését, hogy világítson.

2) Ismételjük meg a fenti lépést még kétszer a sárga és a zöld LED-re is, a kapcsolási rajzhoz hasonlóan.

3) Nevezzük ki a breadboard oldalsó oszlopainak egyikét a közös földelésnek (az ábrán a fekete drótok képviselik). Ide kössük be az ellenállásokat. Ugyanebbe az oszlopba kössük be a Raspberry Pi egyik **GND** jelölésű tüskéjét is egy jumper drót segítségével.

4) A piros LED szabadon levő lábát egy jumperrel kössük össze a **22**-es GPIO tüskével, a sárgát a **27**-sel a zöldet pedig a **17**-sel.

## A kód
Nyissunk meg egy új python fájlt és mentsük el pl. ```led_traffic_lights.py``` név alatt.

### A LED-ek kipróbálása

Miután elkészítettük az áramkört, meg kell írnunk a kódot ami utasítja a Raspberry Pi-t, hogyan viselkedjen amikor a gombot megnyomják.

Először betöltjük a ```gpiozero``` modulból a ```LED``` objektumokat ami lehetővé teszik, hogy a Raspi kommunikáljon a LED-del a **GPIO** tüskéken keresztül. Ezek mellett beolvassuk még a ```sleep``` függvényt is, amit a python várakoztatására használunk majd.

```led_traffic_lights.py```:

In [ ]:
from gpiozero import LED
from time import sleep

Az objektum beolvasása után inicializálunk egy ```LED``` objektumot mindegyik diódának, amiket ```red```-nek, ```amber```-nek és ```green```-nek nevezünk el. Egyben megmondjuk a LED-eknek, hogy a *22*-es, *27*-es és *17*-es számú **GPIO** tüskékre kötöttük.  

```led_traffic_lights.py```:

In [ ]:
from gpiozero import LED
from time import sleep

red = LED(22)
amber = LED(27)
green = LED(17)

Ezután, hogy tesztelhessük, működik-e az áramkörünk LED-es része, egészítsük ki a kódot a következő sorokkal, ami be is kikapcsolja a LED-ket, 3 másodperces késéssel:

```led_traffic_lights.py```:

In [ ]:
from gpiozero import LED
from time import sleep

red = LED(22)
amber = LED(27)
green = LED(17)

red.on()
sleep(3)
red.off()

amber.on()
sleep(3)
amber.off()

green.on()
sleep(3)
green.off()

Mentsük el a fájlt és futtasuk le a kódot a terminálban a ```python led_traffic_lights.py``` paranccsal. Ha minden rendben van, a LED-ek világítanak 3 másodpercig felváltva.

### A villanyrendőr lépésekben

A következő lépésben készítünk egy végtelen ```while``` ciklust, amiben, mint egy igazi útkereszteződésben, a villanyrendőr lámpái ismétlik a működési ciklusukat. 

Kezdjük az egészet avval, hogy még a ```while``` ciklus előtt, a zöld LED-et bekapcsoljuk, a többit pedig ki. Tehát a kezdő állapotban az autók mehetnek.

A ```while``` ciklusban a következő lépéseket kell megvalósítani:

* hagyjuk a zöld LED-et világítani 10 másodpercig: ```sleep(10)```,
* zöldet kikapcsolni és a sárgát bekapcsolni 1 másodpercig,
* sárgát kikapcsolni, pirosan bekapolni és 10 másodpercig hagyni világítani,
* piros mellé a sárgát bekapcsolni 1 másodperc erejéig,
* pirosat és sárgát kikapcsolni majd a zöldet be.

```led_traffic_ligts.py```:

In [ ]:
from gpiozero import LED
from time import sleep

red = LED(22)
amber = LED(27)
green = LED(17)

green.on()
amber.off()
red.off()

while True:
   sleep(10)
   green.off()
   amber.on()
   sleep(1)
   amber.off()
   red.on()
   sleep(10)
   amber.on()
   sleep(1)
   green.on()
   amber.off()
   red.off()

Mentsük el a kódot és futtasuk le ismét. Ha minden rendben van, egy működő villanyrendőr LED-es változatát kell látnunk.   

### TrafficLights használata

A *TrafficLights* egy olyan klassz ami lerövidíti és leegyszerűsíti a villanyrendőr modelljének szoftveres elkészítését. Három bemenő paramétert kell definiálni pl. ```lights = TrafficLights(red=2, amber=3, green=4)``` formában, ahol az argumentumok neveit nem muszáj megadni, de a megadott pin számok a piros, sárga és zöld színek pinjeiként lesznek elmentve. Az így inicializált ```lights``` objektum metódusai között ott lesz a ```lights.red```, ```lights.amber``` és a ```lights.green``` LED objektum, amiket akár külön külön is lehet kontrollálni a LED-ek parancsszavaival.

Evvel a módszerrel pár sorral rövidebbé tehetjük a kódunkat. A három LED-et lényegében egy sorban tudjuk inicializálni. Lényegében egy klasszt hoztunk létre egy specifikus feladat megoldására ahhoz, hogy bizonyos lépéseket gyorsabban és átláthatóbban tudjunk megoldani.

A ```while``` ciklusban ugyanazt csináljuk mint az előző kódban, csak most már a ```lights``` objektumra kell hivatkozni és a hozzá tartozó színekhez. 

Mentsük el ezt a kódot a ```led_traffic_lights1.py``` név alatt és próbáljuk meg tesztelni.

```led_traffic_lights1.py```:

In [ ]:
from gpiozero import TrafficLights
from time import sleep

lights = TrafficLights(22, 27, 17)

lights.green.on()

while True:
    sleep(10)
    lights.green.off()
    lights.amber.on()
    sleep(1)
    lights.amber.off()
    lights.red.on()
    sleep(10)
    lights.amber.on()
    sleep(1)
    lights.green.on()
    lights.amber.off()
    lights.red.off()

Lefuttatva a kódot, ugyanazt kellene tapasztalnunk mint az előző kódnál. 

A kódot lefuttatva mostmár a játékosok nevének kell megjelennie a képernyőn. 

## A projekt tesztelése

Miután összeszereltük az áramkört és a kódot is megírtuk, amit pl. ```led_traffic_lights.py``` név alatt mentettünk el, megnyithatunk a Raspberry Pi operációs rendszerén egy terminált. A terminálban a ```cd 'mappa név'``` paranccsal elnavigálunk abba a mappába, ahova a ```led_traffic_lights.py```-t elmentettük. Ott begépelve a ```python led_traffic_lights.py``` parancsot, letesztelhetjük a programunk működését. Ha minden jól megy akkor a LED-ek a villanyrendőr működési elvét fogják utánozni.

Hibaüzenetek esetén ki kell deríteni mi lehetett a probléma, pl. elgépelés, egy modul hiányzik, sorok megfelelő behúzása, idézőjel lemaradása stb. A hibaüzenet legtöbbször segít abban, hogy melyik sorban találta a hibát és hogy mi volt az. Egy kis gyakorlással bele lehet jönni azok értelmezésébe, valamint interneten is rá lehet keresni a hibaüzenet jelentésére és annak lehetséges elhárítására.

## Mit lehet javítani/továbbfejleszteni?

* Gombnyomásra elindítani a villanyrendőrt és gombnyomásra leállítani.
* Változtatni a világítási idejét a LED-eknek.

Írd meg kommentben, hogy szerinted mivel lehetne még feldobni ezt a kis programot!

## Referencia

1) gpiozero leírás - [https://gpiozero.readthedocs.io/en/stable/recipes.html](https://gpiozero.readthedocs.io/en/stable/recipes.html)

2) LED objektum leírása - [https://gpiozero.readthedocs.io/en/stable/api_output.html#led](https://gpiozero.readthedocs.io/en/stable/api_output.html#led)

3) TrafficLights objektum leírása - [https://gpiozero.readthedocs.io/en/v1.2.0/api_boards.html#trafficlights](https://gpiozero.readthedocs.io/en/v1.2.0/api_boards.html#trafficlights)